In [5]:
import re
import os
import json
import csv
import duckdb as db
# from pydantic import BaseModel
from typing import Any, Dict, List, Tuple, TypedDict, Annotated
# from textwrap import dedent
# from crewai_tools import tool, FileWriterTool
# from crewai import Agent, Crew, Process, Task
# from langchain_ollama.llms import OllamaLLM
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI

# CrewAI

In [6]:
# model = ChatGroq(
#     model="llama3-8b-8192",
#     temperature = 0,
# )

In [7]:
model = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0
)

In [8]:
# model = ChatGoogleGenerativeAI(
#     model='gemini-1.5-pro-exp-0801',
#     temperature=0
# )

In [13]:
def query_list(file_path: str) -> List:
    with open(file_path) as f:
        s = f.read()
    
    return s

In [14]:
q = query_list('sql_files/bn_beneficiario/01_beneficiario.sql')

In [15]:
q

"SELECT \n       BEN.HANDLE                                                                         AS ID_BENEFICIARIO\n      ,FAM.HANDLE                                                                         AS ID_FAMILIA\n      ,CON.HANDLE                                                                         AS ID_CONTRATANTE\n      ,NVL(LOT.HANDLE,0)                                                                  AS ID_CONTRATANTE_LOT\n      ,PLA.HANDLE                                                                         AS ID_PLANO\n      ,(SELECT MAX(BEN_TIT.HANDLE) \n        FROM   SAM_BENEFICIARIO BEN_TIT \n        WHERE  BEN_TIT.FAMILIA   = BEN.FAMILIA \n        AND    BEN_TIT.EHTITULAR = 'S')                                                   AS ID_BENEFICIARIO_RESP\n      ,CASE\n         WHEN CON.LOCALFATURAMENTO = 'C' THEN CPES.HANDLE --CPES\n         WHEN CON.LOCALFATURAMENTO = 'L' THEN LPES.HANDLE --LPES\n         WHEN CON.LOCALFATURAMENTO = 'F' THEN\n           CASE

In [53]:
def query_list(steps: list):
    """ 
    Input a tranformed ktr structure, output a list of queries.
    """
    queries = []  

    for step in T.steps:
        if step.type in ['DBJoin', 'TableInput']:
            queries.append({
                'step_name': step.name,
                'sql_code': step.sql 
            })  
    return queries

In [54]:
list_of_queries = query_list(T.steps)

In [55]:
list_of_queries

[{'step_name': 'BENEFICIÁRIO',
  'sql_code': "SELECT \n       BEN.HANDLE                                                                         AS ID_BENEFICIARIO\n      ,FAM.HANDLE                                                                         AS ID_FAMILIA\n      ,CON.HANDLE                                                                         AS ID_CONTRATANTE\n      ,NVL(LOT.HANDLE,0)                                                                  AS ID_CONTRATANTE_LOT\n      ,PLA.HANDLE                                                                         AS ID_PLANO\n      ,(SELECT MAX(BEN_TIT.HANDLE) \n        FROM   SAM_BENEFICIARIO BEN_TIT \n        WHERE  BEN_TIT.FAMILIA   = BEN.FAMILIA \n        AND    BEN_TIT.EHTITULAR = 'S')                                                   AS ID_BENEFICIARIO_RESP\n      ,CASE\n         WHEN CON.LOCALFATURAMENTO = 'C' THEN CPES.HANDLE --CPES\n         WHEN CON.LOCALFATURAMENTO = 'L' THEN LPES.HANDLE --LPES\n         WHEN CON

In [ ]:
list_of_queries

[{'step_name': 'BENEFICIÁRIO',
  'sql_code': "SELECT \n       BEN.HANDLE                                                                         AS ID_BENEFICIARIO\n      ,FAM.HANDLE                                                                         AS ID_FAMILIA\n      ,CON.HANDLE                                                                         AS ID_CONTRATANTE\n      ,NVL(LOT.HANDLE,0)                                                                  AS ID_CONTRATANTE_LOT\n      ,PLA.HANDLE                                                                         AS ID_PLANO\n      ,(SELECT MAX(BEN_TIT.HANDLE) \n        FROM   SAM_BENEFICIARIO BEN_TIT \n        WHERE  BEN_TIT.FAMILIA   = BEN.FAMILIA \n        AND    BEN_TIT.EHTITULAR = 'S')                                                   AS ID_BENEFICIARIO_RESP\n      ,CASE\n         WHEN CON.LOCALFATURAMENTO = 'C' THEN CPES.HANDLE --CPES\n         WHEN CON.LOCALFATURAMENTO = 'L' THEN LPES.HANDLE --LPES\n         WHEN CON

In [56]:
file_write_tool = FileWriterTool()

In [57]:
sql_analyst = Agent(
    role = "Senior Data Analyst",
    goal = "Analyse and complex sql queries and extract table name vs column name relation from all tables on the querie",
    backstory=dedent(
        """
        You're a highly specialized developed to dissect and understand complex SQL queries,
        you could quickly and accurately extract essential information from intricate SQL statements.
        Your key traits are Analytical prowess, Attention to detail, Vast knowledge of SQL syntax 
        across multiple database systems.
        """
    ),
    llm=model,
    allow_delegation=False
)

In [58]:
report_writer = Agent(
    role = "Senior Business Analyst",
    goal = "Write a document based on the work of analyst",
    backstory = dedent(
        """
        You writing still is well known for clear and effective communication.
        You summarize Pentahor workflow artifacts and sql queries business rules into bullet point contain de most import details.
        """
    ),
    llm=model,
    allow_delegation=False
)

In [69]:
extract_tables_columns = Task(
    description=dedent(
        """
        Analyse this SQL querie {sql_code}.
        Is very important use table name, not its nick name. Then extract tables e columns names.
        Get only table name and column name following this patterns:
        table_name;columns_name
        table1;columnName_n1
        table1;columnName_n2
        table2;columnName_n1
        table2;columnName_n2
        tableN;columnName_n1
        tableN;columnName_n1
        """
    ),
    expected_output="CSV file",
    agent=sql_analyst,
    #callback=lambda result: result_collector.add_result(result)
)

In [70]:
export_tables_columns = Task(
    description = "Get only table name and column.",
    expected_output = "CSV file",
    agent = sql_analyst,
    tools=[file_write_tool],
    context = [extract_tables_columns],
    #output_file='output/{step_name}.csv',
    #create_directory = True,
    #callback=lambda result: result_collector.add_result("export_tables_columns", result)
)

In [71]:
analyze_data = Task(
    description = "Analyze the query {sql_code} and write an analysis.",
    expected_output = "Detailed analysis from text for non technical public.",
    agent = sql_analyst,

)

In [72]:
write_report = Task(
    description=dedent(
        """
        Write an tecnical report from previous analysis, this document will be used 
        to migrate the actual project to cloud enviroment. 
        """
    ),
    expected_output = "Markdown report",
    agent=report_writer,
    context=[analyze_data]
)

In [73]:
# crew = Crew(
#     agents = [sql_analyst, report_writer],
#     tasks = [extract_tables_columns,format_queries,analyze_data,write_report],
#     process = Process.sequential,
#     verbose=1,
#     memory=False,
#     output_log_file="crew.log",
# )

In [74]:
crew = Crew(
    agents = [sql_analyst],
    tasks = [extract_tables_columns],
    process = Process.sequential,
    verbose = 0,
    memory=False,
    output_log_file="crew.log",
)

2024-08-28 14:31:58,897 - 131931064725504 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [75]:
list_of_queries

[{'step_name': 'BENEFICIÁRIO',
  'sql_code': "SELECT \n       BEN.HANDLE                                                                         AS ID_BENEFICIARIO\n      ,FAM.HANDLE                                                                         AS ID_FAMILIA\n      ,CON.HANDLE                                                                         AS ID_CONTRATANTE\n      ,NVL(LOT.HANDLE,0)                                                                  AS ID_CONTRATANTE_LOT\n      ,PLA.HANDLE                                                                         AS ID_PLANO\n      ,(SELECT MAX(BEN_TIT.HANDLE) \n        FROM   SAM_BENEFICIARIO BEN_TIT \n        WHERE  BEN_TIT.FAMILIA   = BEN.FAMILIA \n        AND    BEN_TIT.EHTITULAR = 'S')                                                   AS ID_BENEFICIARIO_RESP\n      ,CASE\n         WHEN CON.LOCALFATURAMENTO = 'C' THEN CPES.HANDLE --CPES\n         WHEN CON.LOCALFATURAMENTO = 'L' THEN LPES.HANDLE --LPES\n         WHEN CON

In [76]:
def convert_to_filename(input_string):
    s = re.sub(r'[^a-zA-Z0-9_\-\.~]', '_', input_string)
    return s.lower()

In [77]:
##self.export_to_csv()

def save_to_csv(data, filename):
    # Split the string by newlines to get rows
    rows = data.split('\n')
    
    # Split each row by semicolons to get columns
    formatted_data = [row.split(';') for row in rows]
    
    # Output filename
    output_filename = f"{'output'}/{filename}.csv"

    # Write the formatted data to a CSV file
    with open(output_filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Column', 'Table'])  # Write header
        writer.writerows(formatted_data)

In [78]:
for query in list_of_queries:
    print(query['step_name'])
    file_name = convert_to_filename(query['step_name'])
    result = crew.kickoff(inputs=query)
    output = extract_tables_columns.output
    save_to_csv(output.raw, file_name)
    #result_collector.add_result('export_tables_columns',output.raw,file_name)

BENEFICIÁRIO
BUSCA MICROSIGA
CONTRATO
FAMILIA PESSOA RESPONSÁVEL
FAMILIA TITULAR RESPONSÁVEL
HANDLE_BENEFICIARIO
LOTAÇÃO
SAM_FAMILIA_TETO_PF
SEM SETOR


# Processa CSVs

In [45]:
db.sql(
"""     
    with todas as (
        select * from 'output/benefici_rio.csv' 
        union all
        select * from 'output/contrato.csv' 
        union all
        select * from 'output/familia_pessoa_respons_vel.csv'
        union all
        select * from 'output/familia_titular_respons_vel.csv'
        union all
        select * from 'output/lota__o.csv'
    )
    select distinct todas.* from todas
    order by todas.Column
""")

┌────────────────────┬────────────────────────────────┐
│       Column       │             Table              │
│      varchar       │            varchar             │
├────────────────────┼────────────────────────────────┤
│ AGENCIA            │ SFN_CONTAFIN_COMPLEMENTO       │
│ AGENCIA            │ SFN_AGENCIA                    │
│ ANOTACAO           │ SAM_FAMILIA_ANOTADM            │
│ ATENDIMENTOATE     │ SAM_BENEFICIARIO               │
│ BAIRRO             │ SAM_ENDERECO                   │
│ BANCO              │ SFN_CONTAFIN_COMPLEMENTO       │
│ BEMESTARESAUDE     │ K_SAM_BENEFICIARIO_APOIOSAUDE  │
│ BEN                │ HANDLE                         │
│ BENEFICIARIO       │ SAM_BENEFICIARIO_EVENTO        │
│ BENEFICIARIO       │ SFN_CONTAFIN                   │
│    ·               │      ·                         │
│    ·               │      ·                         │
│    ·               │      ·                         │
│ TITULAR            │ SAM_BENEFICIARIO_TITULAR 